# Курс Специализация Data Science Дипломный проект. Бриф учебного кейса
## «Модель прогнозирования стоимости жилья для агентства недвижимости»

#### 3 Этап создания модели

* В данной дипломной работе  мы сделали следующее:
* Обработали и отнормировали признаки
* Построим "наивную" модель, предсказывающую цену по общей площади и городу (с ней будем сравнивать другие модели)
* Построим модель при помощи LinearRegression
* Обучим модель на основе случайного леса RandomForestRegressor
* Обучим модель с L1 и L2 регуляризаций ElasticNetCV
* Попытались  запустить модель градиентного бустинга CatBoostRegressor


In [ ]:
pip install optuna

In [ ]:
pip install catboost

In [ ]:
pip install category_encoders

In [ ]:
import random
import numpy as np
import pandas as pd
import sys
import optuna

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn import metrics
from tqdm.notebook import tqdm
from category_encoders import TargetEncoder, CatBoostEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures


# plt
import matplotlib.pyplot as plt
#Увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#Графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline
#Отключим оповещения
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42
TEST_SIZE = 0.2

In [ ]:
df = pd.read_csv('/content/data_model.csv')
display(df.head())
df.info()

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,4.0,Southern Pines,2900,28387,NC,418000,False,single_family_home,2019,True,False,False,True,5.2,2.70
1,For Sale,3.0,Spokane Valley,1947,99216,WA,310000,False,single_family_home,2019,False,False,False,False,4.0,1.01
2,Active,2.0,Mason,3588,50401,IA,244900,False,single_family_home,1970,True,True,False,False,3.8,5.60
3,Other,3.0,Houston,1930,77080,TX,311995,False,single_family_home,2019,True,True,True,False,3.0,0.60
4,For Sale,2.0,Flushing,1300,11354,NY,669000,False,condo,1965,False,False,True,False,2.8,0.30


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               210669 non-null  object 
 1   baths                210669 non-null  float64
 2   city                 210669 non-null  object 
 3   sqft                 210669 non-null  int64  
 4   zipcode              210669 non-null  int64  
 5   state                210669 non-null  object 
 6   target               210669 non-null  int64  
 7   pool_encoded         210669 non-null  bool   
 8   Type                 210669 non-null  object 
 9   Year built           210669 non-null  object 
 10  Heating_encoded      210669 non-null  bool   
 11  Cooling_encoded      210669 non-null  bool   
 12  Parking_encoded      210669 non-null  bool   
 13  fireplace_encoded    210669 non-null  bool   
 14  school_rating _mean  210669 non-null  float64
 15  school_dist_min  

In [ ]:
# Составим список булевых признаков:
bin_features = ['pool_encoded','Heating_encoded','Cooling_encoded','Parking_encoded','fireplace_encoded']

# Составим список категориальных признаков:
cat_features = ['status','city','zipcode','state','Type','Year built']

# Составим список числовых признаков:
num_features = ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']

In [ ]:
# Подсчет количества уникальных значений в каждой категориальной колонке
for col in cat_features:
   unique_values = df[col].nunique()
   print(f"Количество уникальных значений в категориальной колонке {col}: {unique_values}")

Количество уникальных значений в категориальной колонке status: 12
Количество уникальных значений в категориальной колонке city: 1522
Количество уникальных значений в категориальной колонке zipcode: 3962
Количество уникальных значений в категориальной колонке state: 34
Количество уникальных значений в категориальной колонке Type: 12
Количество уникальных значений в категориальной колонке Year built: 205


In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''

    df_output = df_input.copy()
    # Переведем признак zipcode в категориальный
    df_output['zipcode'] = df_output['zipcode'].astype(str)
    # Переведем признак год в категориальный
    df_output['Year built'] = df_output['Year built'].astype(str)
    # Нормализация и логорифмирование данных
    #scaler = MinMaxScaler()
    for column in ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']:
        #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        # Логорифмирование
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)

    # ################### Categorical Features ##############################################################

    ohe_status = OneHotEncoder
    ohe_state = OneHotEncoder
    ohe_Type = OneHotEncoder

    le = LabelEncoder()
    state_label = le.fit_transform(df_output['state'])

    year_le = LabelEncoder()
    year_ord = year_le.fit_transform(df_output['Year built'])

    city_le = LabelEncoder()
    city_label = city_le.fit_transform(df_output['city'])

    zip_le = LabelEncoder()
    zip_label = zip_le.fit_transform(df_output['zipcode'])

    # Adding encoded categorical features to the output dataframe

    df_output['state_label'] = state_label
    df_output['year_ord'] = year_ord
    df_output['city_label'] = city_label
    df_output['zip_label'] = zip_label

    # Dropping original categorical columns
    df_output.drop(['status', 'state', 'Type', 'city', 'zipcode','Year built'], axis=1, inplace=True)

    return df_output

In [ ]:
# Запускаем и проверяем, что получилось
df_encoded = preproc_data(df)
df_encoded.sample(10)

,baths,sqft,target,pool_encoded,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min,state_label,year_ord,city_label,zip_label
163052,0.693148,8.006034,12.936034,False,True,True,False,True,1.386295e+00,9.999995e-07,27,180,899,1358
44170,1.386295,7.971431,12.449019,False,True,True,True,True,2.079442e+00,-3.566735e-01,28,189,1182,2666
197587,1.386295,8.057377,13.345507,False,True,True,True,False,9.999995e-07,-9.430958e-02,25,199,1032,411
137414,0.693148,7.623153,12.487485,True,False,False,True,True,1.193923e+00,-1.863284e-01,5,135,644,910
123956,1.098613,7.259820,12.542541,False,True,True,True,False,1.840550e+00,-3.566735e-01,20,189,727,3101
48620,0.693148,7.406103,11.288531,False,True,True,False,False,1.386295e+00,-2.302575e+00,6,204,5,873
84473,0.693148,7.487174,12.154779,False,True,True,True,True,1.740466e+00,-1.743522e-01,28,186,608,2518
57370,1.098613,8.035279,12.706545,False,True,True,True,True,1.945910e+00,-1.469672e+00,28,188,736,2572
39313,1.098613,8.024207,13.079456,False,False,True,False,False,1.386295e+00,-1.203969e+00,18,133,229,725
26169,1.098613,7.682022,12.487108,False,True,True,True,False,2.041220e+00,3.364730e-01,9,180,703,1814


In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   baths                210669 non-null  float64
 1   sqft                 210669 non-null  float64
 2   target               210669 non-null  float64
 3   pool_encoded         210669 non-null  bool   
 4   Heating_encoded      210669 non-null  bool   
 5   Cooling_encoded      210669 non-null  bool   
 6   Parking_encoded      210669 non-null  bool   
 7   fireplace_encoded    210669 non-null  bool   
 8   school_rating _mean  210669 non-null  float64
 9   school_dist_min      210669 non-null  float64
 10  state_label          210669 non-null  int64  
 11  year_ord             210669 non-null  int64  
 12  city_label           210669 non-null  int64  
 13  zip_label            210669 non-null  int64  
dtypes: bool(5), float64(5), int64(4)
memory usage: 15.5 MB


Разделим датасет

In [ ]:
y = df_encoded.target.values
X = df_encoded.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Обучение модели

## Model 1: Создадим "Naive" модель
Эта модель будет предсказывать среднюю стоимость по общей площади и городу.
C ней будем сравнивать другие модели.

In [ ]:
# Наивная модель
class NaiveModel:
    def __init__(self):
        self.means = None

    def fit(self, X, y):
        X_df = pd.DataFrame(X, columns=['city_label'])
        y_df = pd.DataFrame(y, columns=['target'])
        df = pd.concat([X_df, y_df], axis=1)
        self.means = df.groupby(['city_label'])['target'].mean().reset_index()

    def predict(self, X):
        X = pd.DataFrame(X, columns=['city_label']).copy()
        X['mean'] = np.nan
        for idx, row in self.means.iterrows():
            X.loc[(X['city_label'] == row['city_label']), 'mean'] = row['target']

        X['mean'].fillna(X['mean'].mean(), inplace=True)
        return X['mean'].to_numpy()

naive_model = NaiveModel()
naive_model.fit(X_train, y_train)
y_pred_train = naive_model.predict(X_train)
y_pred_test = naive_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")


Train MSE: 0.43
Test MSE: 0.43
Train MAE: 0.48
Test MAE: 0.48
Train R2: -0.00
Test R2: -0.00


MAE (Средняя абсолютная ошибка, Mean Absolute Error) - это мера ошибки, вычисленная как среднее значение абсолютных значений ошибок. Меньшие значения MAE указывают на лучшую точность модели. В данном случае:

Train MAE: 0.48 - это средняя абсолютная ошибка на обучающей выборке.
Test MAE: 0.48 - это средняя абсолютная ошибка на тестовой выборке.
R^2 (R2 коэффициент детерминации) - статистическая мера, которая показывает, насколько хорошо вариации зависимой переменной объясняются моделью. Значения R^2 (R2) находятся в диапазоне от -∞ до 1. Чем ближе значение R^2 (R2) к 1, тем лучше модель объясняет зависимость между переменными. В данном случае:

Train (R^2) R2: -0.00 - это коэффициент детерминации на обучающей выборке.
Test (R^2) R2: -0.00 - это коэффициент детерминации на тестовой выборке.
Проанализировав результаты, можно сделать следующие выводы:

Значения MSE и MAE, как для обучающей, так и для тестовой выборок, практически одинаковы. Это говорит о том, что модель демонстрирует схожую производительность на обеих выборках и не страдает от явного переобучения или недообучения.

Значения (R^2) R2 близки к нулю, что говорит о том, что модель слабо объясняет зависимость между переменными. Это может свидетельствовать о низкой предсказательной способности модели.

## Model 2: LinearRegression

In [ ]:
# Cоздаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)

# Вычисляем коэффициенты регрессии
model.fit(X_train, y_train)

# Делаем предсказания с помощью модели
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Вычисляем требуемые метрики
mse_train = metrics.mean_squared_error(y_train, y_train_pred)
mse_test = metrics.mean_squared_error(y_test, y_test_pred)
mae_train = metrics.mean_absolute_error(y_train, y_train_pred)
mae_test = metrics.mean_absolute_error(y_test, y_test_pred)
r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.49
Test MSE: 0.50
Train MAE: 0.50
Test MAE: 0.50
Train R2: -0.16
Test R2: -0.18


Среднеквадратическая ошибка (Mean Squared Error, MSE) В данном случае, как для обучающей, так и для тестовой выборки MSE составляет 0.50. Низкие значения MSE указывают на более точные предсказания модели.

Средняя абсолютная ошибка (Mean Absolute Error, MAE) В данном случае MAE составляет 0.50 для обучающей и 0.50 для тестовой выборки. Значения MAE говорят о низкой точности предсказания.

Коэффициент детерминации (R2) В данном случае R2 составляет -0.16 как для обучающей, так и для тестовой выборки, что означает, что модель объясняет 18% дисперсии зависимой переменной.

Результаты модели показывают, что она работает с одинаковой точностью на обучающем и тестовом наборе данных, что свидетельствует о том, что модель не переобучилась. Однако, коэффициент детерминации R2 указывает на то, что есть еще большой потенциал для улучшения модели, поскольку ходят возможны другие альтернативы, такие как увеличение размера выборки, использование других переменных или алгоритмов для еще большей предсказательной точности.

## Model 3: RandomForestRegressor

In [ ]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)

# Предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.01
Test MSE: 0.09
Train MAE: 0.07
Test MAE: 0.18
Train R2: 0.97
Test R2: 0.79


Среднеквадратическая ошибка (MSE) - В данном случае, значение MSE для обучающей выборки составляет 0.01, тогда как для тестовой выборки - 0.09. Такое различие может указывать на небольшое переобучение модели.

Средняя абсолютная ошибка (MAE) -  В данном случае, значения MAE составили 0.07 для обучающей выборки и 0.18 для тестовой выборки. Разница между этими значениями также может указывать на переобучение.

Коэффициент детерминации (R2) -  В данном случае, R2 равен 0.97 для обучающей выборки и 0.79 для тестовой выборки. Эти значения указывают на то, что модель довольно хорошо работает на обучающей выборке и неплохо предсказывает на тестовой выборке, хотя значение R2 на тестовой выборке заметно ниже, что также может указывать на переобучение модели.

В целом, результаты показывают, что модель имеет хорошую производительность на обучающей выборке, но на тестовой выборке результаты хуже, что может свидетельствовать о переобучении.

## Model 4: ElasticNetCV

In [ ]:
# Создаем и тренируем модель ElasticNetCV с кросс-валидацией по 5 фолдам
model_el = ElasticNetCV(cv=5, random_state=RANDOM_SEED)
model_el.fit(X_train, y_train)

# Предсказания для обучающей и тестовой выборок
y_train_pred = model_el.predict(X_train)
y_test_pred = model_el.predict(X_test)

# MSE
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# R2
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.37
Test MSE: 0.38
Train MAE: 0.45
Test MAE: 0.45
Train R2: 0.12
Test R2: 0.11


MSE (Mean Squared Error) - В данном случае обучающая выборка имеет значение 0.37, а тестовая - 0.38. Эти числа близки, что означает, что модель не переобучилась, но в то же время обобщающая способность модели может быть не очень хорошей из-за высоких значений ошибок.

MAE (Mean Absolute Error) - В данном случае средняя абсолютная ошибка составляет 0.45 как для обучающей, так и для тестовой выборки. Это означает, что в среднем модель ошибается на 0.45 при предсказаниях.

R2 (коэффициент детерминации) -  В данном случае R2 равен 0.12 для обучающей выборки и 0.11 для тестовой выборки. Эти значения невысоки, что говорит о плохом качестве модели и низкой способности объяснить изменчивость данных.

Вывод: по представленным результатам видно, что модель имеет среднюю ошибку и невысокий коэффициент детерминации. Это означает, что модель не является оптимальной.

## Model 5: CatBoostRegressor

CatBoostRegressor позволяет работать с необработанными категориальными данными, в отличие от некоторых других алгоритмов машинного обучения, которые требуют предварительной обработки данных, такой как кодирование категориальных переменных в числовые значения.

In [ ]:
def log_data(df_input):
    '''includes several functions to pre-process the predictor data.'''

    df_output = df_input.copy()
    # Переведем признак зип кода в категориальный
    df_output['zipcode'] = df_output['zipcode'].astype(str)
    # Переведем признак год в категориальный
    df_output['Year built'] = df_output['Year built'].astype(str)
    # Нормализация данных и логорифмирование
    scaler = MinMaxScaler()
    for column in num_features:
        #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        # Логорифмирование
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)
    return df_output

In [ ]:
# Запускаем и проверяем, что получилось
df_log = log_data(df)
df_log.sample(5)

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
182,For Sale,0.693148,Fort Worth,7.696213,76105,TX,12.206068,False,single_family_home,2019,True,False,True,False,1.193923,-0.693145
47534,For Sale,1.098613,Miami,7.676010,33179,FL,12.886641,False,single_family_home,2013,True,True,True,False,1.223776,0.173954
87293,For Sale,0.693148,Miami,7.038784,33133,FL,12.818552,True,townhouse,1982,True,True,True,False,0.470004,-0.755020
193635,For Sale,1.098613,Knoxville,8.115222,37931,TN,12.721587,False,single_family_home,2007,True,False,True,True,1.791760,0.378437
136192,For Sale,0.693148,Brooklyn,7.094235,11214,NY,13.499429,False,condo,2006,False,False,False,False,2.079442,-1.347070


In [ ]:
y = df_log.target.values
X = df_log.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
param_grid = {
    'iterations': [100, 300, 500],
    'learning_rate': [0.01, 0.03, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5],
}
cb_model = CatBoostRegressor(random_seed=RANDOM_SEED, silent=True)
grid_search = GridSearchCV(estimator=cb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train, cat_features=cat_features)
best_params = grid_search.best_params_
best_cb_model = CatBoostRegressor(iterations=best_params['iterations'], learning_rate=best_params['learning_rate'], depth=best_params['depth'], l2_leaf_reg=best_params['l2_leaf_reg'], random_seed=RANDOM_SEED, silent=True)
best_cb_model.fit(X_train, y_train, cat_features=cat_features)

y_train_pred = best_cb_model.predict(X_train)
y_test_pred = best_cb_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits


KeyboardInterrupt: 

# #Ошибка в коде

In [ ]:
data = {'Metric': ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2'],
        'NaiveModel': [0.43, 0.43, 0.48, 0.48, -0.00, -0.00],
        'LinearRegression': [0.49, 0.50, 0.50, 0.50, -0.16, -0.18],
        'RandomForestRegressor': [0.01, 0.09, 0.07, 0.18, 0.97, 0.79],
        'ElasticNetCV': [0.37, 0.38, 0.45, 0.45, 0.12, 0.11],
      }

df_metric = pd.DataFrame(data)
df_metric.head(6)


,Metric,NaiveModel,LinearRegression,RandomForestRegressor,ElasticNetCV
0,Train MSE,0.43,0.49,0.01,0.37
1,Test MSE,0.43,0.50,0.09,0.38
2,Train MAE,0.48,0.50,0.07,0.45
3,Test MAE,0.48,0.50,0.18,0.45
4,Train R2,-0.00,-0.16,0.97,0.12
5,Test R2,-0.00,-0.18,0.79,0.11


На основе этих результатов, RandomForestRegressor кажется наилучшей моделью, так как она имеет наименьший показатель ошибок MSE и MAE, вместе с высоким коэффициентом детерминации R2 (0.79) на тестовых данных. Однако, стоит учесть переобучение случайного леса, так как разрыв между показателями на тренировочных и тестовых наборах является существенным.

CatBoostRegressor не получилось реализовать(

In [ ]:
import pickle

In [ ]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('myfile.pkl', 'wb') as output:
    pickle.dump(model_el, output)